In [ ]:
import pymssql
import pandas as pd
import numpy as np
import os
import re
import joblib
import sys 
import DSI_Capstone_Steemit
# from DSI_Capstone_Steemit.utils.utils import(
#     load_data_and_description,
#     load_raw
# )

# load_data_and_description(data_type = 'tfidf')
data_directory = '../data/'
import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline

In [ ]:
authors = pd.read_csv(os.path.join(data_directory,'accounts.csv'))

In [ ]:
from sklearn.preprocessing import FunctionTransformer
transformer = FunctionTransformer(np.log1p)
reputation_scaled = transformer.fit_transform(authors['reputation'])[0]
reputation_scaled = reputation_scaled - min(reputation_scaled) + 0.5

authors['reputation scaled'] = reputation_scaled

In [ ]:
def load_raw_cleaned():
    posts_path = os.path.join(data_directory, 'posts_raw_cleaned',
                              'posts_raw_cleaned.csv')
    df_posts = pd.read_csv(posts_path)
    return df_posts

raw = load_raw_cleaned()

In [ ]:
raw = raw[(raw['depth'] == 0)]

In [ ]:
idx_popular_athors = raw['author'].isin(authors['name'].str.lower().values)

In [ ]:
authors = authors.ix[0:5000,:]

In [ ]:
connections_list = []
for i,name in enumerate(authors.ix[:,'name'].values):
    idx_name = raw['body mentions'].str.count(name) > 0
    connections = raw.ix[idx_name,'author'].value_counts().reset_index()
    connections.columns = ['target','weight']
    connections['source'] = name
    connections_list.append(connections)


In [ ]:
# Node size can be the 

In [ ]:
df_network_x = pd.concat(connections_list)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
mm = MinMaxScaler(feature_range=(0,15))

In [ ]:
scaled_lookup = authors.set_index('name').to_dict()['reputation scaled']

In [ ]:
df_network_x['node_size'] = df_network_x['source'].replace(scaled_lookup)

In [ ]:
import seaborn as sns
%matplotlib inline

In [ ]:
df_network_x.to_csv('../data/networkx/df_network.csv')